<a href="https://colab.research.google.com/github/respinar1/GoogleColaboratory/blob/main/BudgetingProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metadata

In [ ]:
import sqlite3
import seaborn as sn
import pandas as pd
import numpy as np
import datetime as dt
print ('Done!')

Done!


Create the Databases using SQLite

In [ ]:
conn = sqlite3.connect('Budget.db')
c = conn.cursor()

In [ ]:
c.execute('''CREATE TABLE IF NOT EXISTS Users
             ([user_id] INTEGER NOT NULL PRIMARY KEY,
             [F_name] TEXT NOT NULL, 
             [L_name] TEXT NOT NULL, 
             [User_name] TEXT NOT NULL, 
             [Pword] BLOB NOT NULL)''')
print ('Done!')

Done!


In [ ]:
c.execute('''CREATE TABLE IF NOT EXISTS Info
             ([user_id] INTEGER NOT NULL, 
             [name] TEXT NOT NULL, 
             [amount] REAL NOT NULL, 
             [type] TEXT NOT NULL, 
             [frequency] TEXT NOT NULL, 
             [date_due] TEXT NOT NULL, 
             [category] TEXT DEFAULT NULL,
             [priority] INTEGER DEFAULT NULL,
             [date_entered] DATE NOT NULL)''')
print ('Done!')

Done!


In [ ]:
c.execute('''CREATE TABLE IF NOT EXISTS Budget
             ([user_id] INTEGER NOT NULL,
             [amount] INTEGER NOT NULL,
             [goal_date] DATE  NOT NULL
             )''')
print ('Done!')

Done!


Start Program Function

In [ ]:
def start():
  loginsignup()

# Login/Signup

This will allow users to create a new account that is stored on a database or log into a previous account thats already saved on the database

In [ ]:
def loginsignup():
  #Introduce the User
  print("Returning user? (Y/N)")
  
  loop = True
  #(index is used to allow users to sign up and immediately try to login)
  index = 0
  while loop == True:
    
    #Retrieve user input based on prompt
    if index == 0:
      ans = input().upper()
    else:
      ans = 'Y'
    
    #If user answers yes we will close out of the while loop and prompt the user
    #for login information that is compared to our database
    if ans == 'Y':
      
      while loop == True:
        user_name = input("username: ")
        p_wrd = input("password: ")
      
        c.execute('SELECT user_id FROM Users WHERE User_name = ? AND Pword = ?', (user_name, p_wrd))
        check = c.fetchall()
      
        if len(check) != 0:
          loop = False
          c.execute('SELECT F_name, L_name FROM Users')
          name = c.fetchall()
        
          for x in name:
            f_name = x[0]
            l_name = x[1]
          print('\nWelcome '+f_name+'!\n')
        
          #Call the menu function and pass our user_id to be used for our database
          for x in check:
            user_id = x[0]
          menu(user_id)
      
        else:
          print('incorrect username or password') 
    
    #if user answers no we will close out of the while loop and prompt the user
    #to signup and input information to our database
    elif ans == 'N':
      
      index = 1
      
      while loop == True:
        c.execute('SELECT user_id FROM Users')
        id_check = c.fetchall()
        
        #Checks to see if database is empty
        if len(id_check) == 0:
          user_name = input('Enter a Username: ')
          loop = False
        
        else:
          user_name = input('Enter a Username: ')
          c.execute('SELECT User_name FROM Users')
          user_name_db = c.fetchall()
          for x in user_name_db:
            for y in x:
              if user_name == y:
                print('Username is taken, try again')
              else:
                loop = False
      
      loop = True
      first_name = input('Enter your first name: ')
      last_name = input('Enter your Last name: ')
      
      #making sure that the user types in the correct password
      password = False
      while password == False:
        p_word = input('Enter a Password: ')
        confirm_p_wrd = input("Confirm Password: ")

        if p_word == confirm_p_wrd:
          password = True
        
        else:
          print("password are not the same, try again.")
      
      #stores user information onto our database
      c.execute('INSERT INTO Users (F_name, L_name, User_name, Pword) VALUES (?,?,?,?)', (first_name, last_name, user_name, p_word))

    elif ans == 'C':
      print("Good Bye")
      return
    else:
      print("incorrect input, please try again.")

# Menu

The available options the user can choose from

In [ ]:
def menu(user_id):
  
  print('What would you like to do today?\n')
  menu_options = {1:"Account Info", 2:"Show Details", 3:"Budget", 0:"Sign Out"}
  
  loop = True
  while loop == True:
  
    print(menu_options)
    ans = int(input())
    
    if ans in menu_options:
      loop = False
    
      if ans == 0: 
        print("Signed Out")
        loginsignup()
      
      elif ans == 1:
        print("You have selected "+ menu_options[ans])
        account_info(user_id)
        return
      
      elif ans == 2:
        print("You have selected "+ menu_options[ans])
        show_details(user_id)
        return
      
      else:
        print("You have selected "+ menu_options[ans])
        #budget(user_id)
        return
    
    else:
      print("Sorry not an option")



# Account

Add or Remove Income/Expense Data

In [ ]:
def account_info(user_id):

  account_options = {1: "Add Income", 2: "Delete Income", 3: "Add Expense", 4: "Delete Expense", 0: "Go Back"}

  loop = True
  while loop == True:
    print(account_options)
    ans = int(input())

    if ans in account_options:
      loop = False

      if ans == 1 or ans == 2:
        print("You have selected to "+ account_options[ans])
        income(ans, user_id)
      
      elif ans == 3 or ans == 4:
        print("You have selected to "+ account_options[ans])
        expense(ans, user_id)
      
      else:
        menu(user_id)
    
    else:
      print("Sorry not an option")

Income/Expense

In [ ]:
def income(ans, user_id):
  if ans == 1:
    add_income(user_id)
  else:
    remove_income(user_id)

In [ ]:
def expense(ans, user_id):
  if ans == 3:
    add_expense(user_id)
  else:
    remove_expense(user_id)

Adding Income Information

In [ ]:
def add_income(user_id):

  #account type is credit
  account = 'Credit'
  loop = True
  while loop == True:
    print("Please enter the following information/n")
    
    #Name of income
    name = input("Name of Income Source: ")
    #Income amount
    amount = int(input("Income Amount: "))

    #How frequent is the income
    index = True
    while index == True:
      frequency_options = {1:"Weekly", 2:"Bi-Weekly", 3:"Monthly", 4:"Annually"}
      print(frequency_options)
      ans = int(input("How often: "))
      if ans in frequency_options:
        frequency = frequency_options[ans]
        index = False
      else:
        print("Not an option, try again")
    
    #Date payment is released
    date = input("Enter pay date (MM-DD-YYYY): ")
    
    #categorizing the income as passive or active
    index = True
    while index == True:
      category_options = {1:"Passive Income", 2:"Active Income"}
      print(category_options)
      ans = int(input("Type of Income: "))
      if ans in category_options:
        category = category_options[ans]
        index = False
      else:
        print("Not an option, try again")

    #current time entered
    x = dt.datetime.now()
    t = x.strftime("%m-%d-%Y")

    c.execute('INSERT INTO Info (user_id, name, amount, type, frequency, date_due, category, date_entered) VALUES (?,?,?,?,?,?,?,?)', (user_id, name, amount, account, frequency, date, category, t))
    print("Successfully added "+name)
    
    ans = input("Would you like to add another? (Y/N)").upper()
    
    index = True
    while index == True:
      if ans == 'Y':
        index = False
      elif ans == 'N':
        loop = False
        index = False
        menu(user_id)
      else:
        print("Sorry not an option")

Remove Income Information

In [ ]:
def remove_income(user_id):
  print("Which one would you like to delete?\n")
  
  x = str(user_id)
  c.execute('SELECT name FROM Info WHERE user_id = ? and type = "Credit"', x)
  y = c.fetchall()
  
  if len(y) == 0:
    print("Nothing to delete")
    menu(user_id)
  
  else:
    index = 0
    for names in y:
      index += 1
      print(str(index)+'. '+names[0])
    
  
    loop = True
    while loop == True:
      ans = int(input())
      if ans <= index and ans > 0:
        loop = False
      else:
        print("Not a valid input")
  
    index = 0
    for names in y:
      index += 1
      if index == ans:
        name = names[0]
      else:
        continue
  
    print("Would you like to remove: " +name+ "?\n")
  
    loop = True
    while loop == True:
      ans = input("(Y/N)").upper()
      if ans == 'Y':
        c.execute('DELETE FROM Info WHERE name = ? AND user_id = ? AND type = "Credit"', (name, x))
        loop = False
        print("Successfully removed "+name)
        menu(user_id)
      elif ans == 'N':
        cont = True
        while cont == True:
          ans = input("Would you like to continue removing an income?(Y/N)").upper()
          if ans == 'N':
            cont = False
            loop = False
            menu(user_id)
          elif ans == 'Y':
            continue
          else:
            print('Invalid answer')
      else:
        print("Invalid answer")
  

Add Expenses

In [ ]:
def add_expense(user_id):

  #account type is Debit
  account = 'Debit'
  loop = True
  while loop == True:
    print("Please enter the following information/n")
    
    #Name of income
    name = input("Name of Expense: ")
    #Income amount
    amount = int(input("Expense Amount: "))

    #How frequent is the income
    index = True
    while index == True:
      frequency_options = {1:"Weekly", 2:"Bi-Weekly", 3:"Monthly", 4:"Annually"}
      print(frequency_options)
      ans = int(input("How often: "))
      if ans in frequency_options:
        frequency = frequency_options[ans]
        index = False
      else:
        print("Not an option, try again")
    
    #Date payment is released
    date = input("Enter pay date (MM-DD-YYYY): ")
    
    #categorizing the income as passive or active
    index = True
    while index == True:
      category_options = {1:"Utility", 2:"Subscription", 3:"Debt", 4:"Misc."}
      print(category_options)
      ans = int(input("Type of Expense: "))
      if ans in category_options:
        category = category_options[ans]
        index = False
      else:
        print("Not an option, try again")

    #How important is this expense
    index = True
    while index == True:
      priority_options = {1:"Important", 2:"Semi-Important", 3:"Not Important"}
      print(priority_options)
      ans = int(input("How essential is the expense? "))
      if ans in priority_options:
        priority = priority_options[ans]
        index = False
      else:
        print("Not an option, try again")

    #current time entered
    x = dt.datetime.now()
    t = x.strftime("%m-%d-%Y")

    c.execute('INSERT INTO Info (user_id, name, amount, type, frequency, date_due, category, priority, date_entered) VALUES (?,?,?,?,?,?,?,?,?)', (user_id, name, amount, account, frequency, date, category, priority, t))
    print("Successfully added "+name)
    
    ans = input("Would you like to add another? (Y/N)").upper()
    
    index = True
    while index == True:
      if ans == 'Y':
        index = False
      elif ans == 'N':
        loop = False
        index = False
        menu(user_id)
      else:
        print("Sorry not an option")

Delete Expense

In [ ]:
def remove_expense(user_id):
  print("Which one would you like to delete?\n")
  
  x = str(user_id)
  c.execute('SELECT name FROM Info WHERE user_id = ? AND type = "Debit"', x)
  y = c.fetchall()
  
  if len(y) == 0:
    print("Nothing to delete")
    menu(user_id)
  
  else:
    index = 0
    for names in y:
      index += 1
      print(str(index)+'. '+names[0])
    
  
    loop = True
    while loop == True:
      ans = int(input())
      if ans <= index and ans > 0:
        loop = False
      else:
        print("Not a valid input")
  
    index = 0
    for names in y:
      index += 1
      if index == ans:
        name = names[0]
      else:
        continue
  
    print("Would you like to remove: " +name+ "?\n")
  
    loop = True
    while loop == True:
      ans = input("(Y/N)").upper()
      if ans == 'Y':
        c.execute('DELETE FROM Info WHERE name = ? and user_id = ? AND type = "Debit"', (name, x))
        loop = False
        print("Successfully removed "+name)
        menu(user_id)
      elif ans == 'N':
        cont = True
        while cont == True:
          ans = input("Would you like to continue removing an expense?(Y/N)").upper()
          if ans == 'N':
            cont = False
            loop = False
            menu(user_id)
          elif ans == 'Y':
            continue
          else:
            print('Invalid answer')
      else:
        print("Invalid answer")
  

# Details

Shows the user's account information

In [ ]:
def show_details(user_id):
  details_options = {1:"Show Income Information", 2:"Show Expense Information", 3:"Show All", 0:"Go Back"}
  
  loop = True
  while loop == True:
    print(details_options)
    ans = int(input("\nWhat would you like to see? "))

    if ans in details_options:
        loop = False
        if ans == 1:
            display(pd.read_sql('SELECT * FROM Info WHERE user_id = :id AND type = :type', conn, params={'id':user_id, 'type':"Credit"}, index_col = "user_id"))
        elif ans == 2:
            display(pd.read_sql('SELECT * FROM Info WHERE user_id = :id AND type = :type', conn, params={'id':user_id, 'type':"Debit"}, index_col = "user_id"))
        elif ans == 3:
            display(pd.read_sql('SELECT * FROM Info WHERE user_id = :id', conn, params={'id':user_id}, index_col = "user_id"))
        
        else:
            menu(user_id)
    else:
      print("Invalid input, try again")
    
    loop = True
    while loop == True:
      ans = input("Would you like to do something else? (Y/N)").upper()

      if ans == 'Y':
        loop = False
        show_details(user_id)
      elif ans == 'N':
        loop = False
        menu(user_id)
      else:
        print("Invalid input")

# Budget

Add or view budget based on Account information (hardest thing here)

In [ ]:
def budget(user_id):
    budget_options = {1:"Show Budget", 2:"Create a Budget", 3:"Edit Budget", 0:"Go Back"}

    loop = True
    while loop == True:
        ans = int(input("\nWhat would you like to do today? "))
        if ans in budget_options:
            loop = False
            if ans == 1:
                show_budget(user_id)
            elif ans == 2:
                create_budget(user_id)
            elif ans == 3:
                edit_budget(user_id)
            else:
                menu(user_id)
        else:
            print("Invalid input, try again")
    
    loop = True
    while loop == True:
      ans = input("Would you like to do something else? (Y/N)").upper()

      if ans == 'Y':
        loop = False
        show_details(user_id)
      elif ans == 'N':
        loop = False
        menu(user_id)
      else:
        print("Invalid input")

In [ ]:
def create_budget(user_id):
    print("In order to create a budget you must answer the following questions")
    amount = float(input("\nHow much do you want to save? "))
    date = input("\nBy when do you want to save that amount? ")

    c.execute('INSERT INTO Budget (amount, goal_date) VALUES (?,?)',(amount, date))

    print("I want to save " + str(amount) + " dollars by " + date +" .")
    print("This will be your financial goal")
    budget(user_id)

In [ ]:
def edit_budget(user_id):
    
    print("Your current financial goal is:")
    c.execute('SELECT * FROM budget WHERE user_id = ?', amount)
    info = c.fetchall()
    for x in info:
        amount = x[0]
        date = x[0]
    
    print("What would you like your new goal to be?")
    amount = float(input("\nHow much do you want to save? "))
    date = input("\nBy when do you want to save that amount? ")

    c.execute('INSERT INTO Budget (amount, goal_date) VALUES (?,?)',(amount, date))

    print("I want to save " + str(amount) + " dollars by " + date +" .")
    print("This will be your new financial goal")
    budget(user_id)

In [ ]:
def show_budget(user_id):
    print("W.I.P Sorry")
    budget(user_id)

# Running the Program

In [ ]:
start()

Returning user? (Y/N)
n
Enter a Username: espinar
Enter your first name: ryan
Enter your Last name: espinal
Enter a Password: project
Confirm Password: project
username: espinar
password: project

Welcome ryan!

What would you like to do today?

{1: 'Account Info', 2: 'Show Details', 3: 'Budget', 0: 'Sign Out'}
2
You have selected Show Details
{1: 'Show Income Information', 2: 'Show Expense Information', 3: 'Show All', 0: 'Go Back'}

What would you like to see? 1


,name,amount,type,frequency,date_due,category,priority,date_entered
user_id,,,,,,,,


Would you like to do something else? (Y/N)n
What would you like to do today?

{1: 'Account Info', 2: 'Show Details', 3: 'Budget', 0: 'Sign Out'}
1
You have selected Account Info
{1: 'Add Income', 2: 'Delete Income', 3: 'Add Expense', 4: 'Delete Expense', 0: 'Go Back'}
2
You have selected to Delete Income
Which one would you like to delete?

Nothing to delete
What would you like to do today?

{1: 'Account Info', 2: 'Show Details', 3: 'Budget', 0: 'Sign Out'}
1
You have selected Account Info
{1: 'Add Income', 2: 'Delete Income', 3: 'Add Expense', 4: 'Delete Expense', 0: 'Go Back'}
4
You have selected to Delete Expense
Which one would you like to delete?

Nothing to delete
What would you like to do today?

{1: 'Account Info', 2: 'Show Details', 3: 'Budget', 0: 'Sign Out'}
1
You have selected Account Info
{1: 'Add Income', 2: 'Delete Income', 3: 'Add Expense', 4: 'Delete Expense', 0: 'Go Back'}
1
You have selected to Add Income
Please enter the following information/n
Name of Income Source

,name,amount,type,frequency,date_due,category,priority,date_entered
user_id,,,,,,,,
1,Job,600.0,Credit,Weekly,8/28/21,Active Income,None,08-28-2021


Would you like to do something else? (Y/N)n
What would you like to do today?

{1: 'Account Info', 2: 'Show Details', 3: 'Budget', 0: 'Sign Out'}
1
You have selected Account Info
{1: 'Add Income', 2: 'Delete Income', 3: 'Add Expense', 4: 'Delete Expense', 0: 'Go Back'}
3
You have selected to Add Expense
Please enter the following information/n
Name of Expense: rent
Expense Amount: 500
{1: 'Weekly', 2: 'Bi-Weekly', 3: 'Monthly', 4: 'Annually'}
How often: 3
Enter pay date (MM-DD-YYYY): 09-01-2021
{1: 'Utility', 2: 'Subscription', 3: 'Debt', 4: 'Misc.'}
Type of Expense: 1
{1: 'Important', 2: 'Semi-Important', 3: 'Not Important'}
How essential is the expense? 1
Successfully added rent
Would you like to add another? (Y/N)n
What would you like to do today?

{1: 'Account Info', 2: 'Show Details', 3: 'Budget', 0: 'Sign Out'}
2
You have selected Show Details
{1: 'Show Income Information', 2: 'Show Expense Information', 3: 'Show All', 0: 'Go Back'}

What would you like to see? 2


,name,amount,type,frequency,date_due,category,priority,date_entered
user_id,,,,,,,,
1,rent,500.0,Debit,Monthly,09-01-2021,Utility,Important,08-28-2021


Would you like to do something else? (Y/N)y
{1: 'Show Income Information', 2: 'Show Expense Information', 3: 'Show All', 0: 'Go Back'}

What would you like to see? 3


,name,amount,type,frequency,date_due,category,priority,date_entered
user_id,,,,,,,,
1,Job,600.0,Credit,Weekly,8/28/21,Active Income,None,08-28-2021
1,rent,500.0,Debit,Monthly,09-01-2021,Utility,Important,08-28-2021


Would you like to do something else? (Y/N)n
What would you like to do today?

{1: 'Account Info', 2: 'Show Details', 3: 'Budget', 0: 'Sign Out'}
0
Signed Out
Returning user? (Y/N)
c
Good Bye
